In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('dataset/Project Dataset/sales_data.csv')

In [3]:
data['Date'] = pd.to_datetime(data['Date'], dayfirst = True)
data = data.sort_values(by = 'Date')
